<a href="https://colab.research.google.com/github/Sakutwf/ProyectoFinalMachineLearning/blob/main/Proyecto_Final_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La contaminación atmosférica es una de las preocupaciones ambientales más importantes en muchos países del mundo, y se ve fuertemente afectada por las condiciones meteorológicas. En este proyecto se generarán pronósticos para la calidad del aire disponibles para la ciudad de Curicó.

In [3]:
#Importación de librerías

import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from fbprophet import Prophet
import holidays
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM



In [ ]:
#Extracción de la base de datos de calidad del aire de la estación meteorológica ubicada en Curicó.

url = "https://sinca.mma.gob.cl/cgi-bin/APUB-MMA/apub.tsindico2.cgi?outtype=xcl&macro=./RVII/709/Cal/PM25//PM25.horario.horario.ic&from=120707&to=211130&path=/usr/airviro/data/CONAMA/&lang=esp&rsrc=&macropath="
csv = pd.read_csv(url, error_bad_lines=False, sep=";", na_values='?', na_filter=True, engine='python')

Se realizará un preprocesamiento se datos con el fin de permitir un trabajo más claro y fácil. Se detallan las acciones paso a paso:

In [ ]:
#De ahora en adelante los datos se llamarán desde la variable df
df = csv.copy()

La primera columna que se tratará es la de Fecha

In [ ]:
#transformando a datos de tipo String
df['FECHA (YYMMDD)'] = df['FECHA (YYMMDD)'].astype(str)

#La fecha se separará en día, mes y año, para ello, en primer lugar, se extraerá la columna de interés
auxDate = df['FECHA (YYMMDD)']

#Se extraen los datos de día, mes y año para dar nuevo formato
year = auxDate.str[:2]
month = auxDate.str[2:4]
day = auxDate.str[4:]

#Se da nuevo formato
df['FECHA (YYMMDD)'] = year+"/"+month+"/"+day

La siguiente columna a modificar es la de Hora

In [ ]:
#Así como en la columna anrerior, se transforma a tipo String y se extrae la columna
df['HORA (HHMM)'] = df['HORA (HHMM)'].astype(str)
auxHour = df['HORA (HHMM)']

#Para transformar el formato de la columna de horas será necesario realizar verificaciones en cada celda para la tranformación
#Para ello se creará una lista auxiliar que contendra los nuevos valores para luego asignarlos a la columna definitiva
auxList = []


